<a href="https://colab.research.google.com/github/Komsomolochka/data-analysis-applications/blob/main/text_analysis/text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [2]:
def write_answer(file_name, *answer):
    print(*answer, file=open(file_name, "w"))

Загрузите датасет. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [3]:
data = pd.read_csv('/content/SMSSpamCollection.txt', sep='	', header=None)

In [4]:
data

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [5]:
texts = list(data[1])

In [7]:
target = list(map(lambda x: 0 if x == 'ham' else 1, data[0]))

Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [10]:
pipeline = Pipeline(
    [("counter", CountVectorizer()),
     ("classifier", LogisticRegression(random_state=2))]
    )

In [11]:
result = cross_val_score(pipeline, texts, target, scoring="f1", cv=10)

In [13]:
round(result.mean(),1)

0.9

In [14]:
write_answer('q5.txt', 0.9)

А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

 "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [15]:
X_test = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"
]

In [16]:
lr_model = pipeline.fit(texts, target)

In [24]:
answer_2 = lr_model.predict(X_test)

In [25]:
print(*answer_2)

1 1 0 0 0


In [26]:
write_answer('q6.txt', *answer_2)

 Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [27]:
ngrams=[(2,2), (3,3), (1,3)]

In [28]:
list_result=[]
for ngram in ngrams:
  pipeline = Pipeline([("counter", CountVectorizer(ngram_range=ngram)), ("classifier", LogisticRegression())])
  res = cross_val_score(pipeline, texts, target, scoring="f1", cv=10)
  list_result.append(round(res.mean(),2))
print(*list_result)

0.82 0.72 0.92


In [29]:
write_answer('q7.txt', *list_result)

Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.
 По какой-то причине  обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно  посчитайте частоты слов и обучите классификатор.  

In [52]:
from pandas.core.common import random_state
list_result = []
for ngram in ngrams:
    X = CountVectorizer(ngram_range=ngram).fit_transform(texts)
    result = cross_val_score(MultinomialNB(), X, target, scoring="f1", cv=10)
    list_result.append(round(result.mean(), 2))
print(*list_result)

0.65 0.38 0.89


In [53]:
write_answer('q8.txt', *list_result)

Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [43]:
pipeline = Pipeline([("counter", TfidfVectorizer()), ("classifier", LogisticRegression())])
result = cross_val_score(pipeline, texts, target, scoring="f1", cv=10)
answer = round(result.mean(),2)
print(answer)

0.88


In [45]:
write_answer('q9.txt', -1)